In [40]:
#main
import pandas as pd
import numpy as np

#plot
from matplotlib import pyplot as plt
import seaborn as sns

#dop
import re
import pickle
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()
from IPython.display import display
from datetime import datetime, timedelta, date, time

#prepprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

#ML
from sklearn.linear_model import LogisticRegression

#metrics 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

# option
pd.set_option("display.max_columns", 300)
plt.rcParams['figure.figsize'] = (15, 10)

PATH_TRAIN = 'data/train.csv'
PATH_TASK = 'data/test.csv'
TARGET_COL = 'target'

In [2]:
time_col = ['time' + str(i) for i in range(1, 11)]
sites = ['site' + str(i) for i in range(1,11)]

train = pd.read_csv(PATH_TRAIN, parse_dates=time_col, index_col='session_id')
task = pd.read_csv(PATH_TASK, parse_dates=time_col, index_col='session_id')
y_train = train[[TARGET_COL]]

print(train.shape, task.shape)
print('-'*50)
train.info()
train.head()

(253561, 21) (82797, 20)
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   site1   253561 non-null  int64         
 1   time1   253561 non-null  datetime64[ns]
 2   site2   250098 non-null  float64       
 3   time2   250098 non-null  datetime64[ns]
 4   site3   246919 non-null  float64       
 5   time3   246919 non-null  datetime64[ns]
 6   site4   244321 non-null  float64       
 7   time4   244321 non-null  datetime64[ns]
 8   site5   241829 non-null  float64       
 9   time5   241829 non-null  datetime64[ns]
 10  site6   239495 non-null  float64       
 11  time6   239495 non-null  datetime64[ns]
 12  site7   237297 non-null  float64       
 13  time7   237297 non-null  datetime64[ns]
 14  site8   235224 non-null  float64       
 15  time8   235224 non-null  datetime64[ns]
 16

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,3846.0,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,39.0,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,782.0,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,178.0,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [4]:
def start_in_seconds(x):
    if type(x) != pd._libs.tslibs.nattype.NaTType:
        return pd.to_timedelta(str(x.time())) // np.timedelta64(1, 's')
    else:
        return -1


def feature_engineering_time(df,
                             time_col,
                             target=True,
                             time_max=66054,
                             time_min=27444
                            ):
    df_time = df[time_col].copy()
    
    print('Обработка: mounth')
    df_time['month'] = df_time[time_col[0]].progress_apply(lambda x: x.date().month)
    
    print('Обработка: weekday')
    df_time['weekday'] = df_time[time_col[0]].progress_apply(lambda x: x.weekday()) 
    
    print('Обработка: time1')
    df_time[time_col[0] + '_sec'] = df_time[time_col[0]].progress_apply(start_in_seconds)
    
    print('Обработка: Разницы')
    col_diff = []
    for i in tqdm(range(len(time_col) - 1)):
        df_time[time_col[i + 1]+'-'+time_col[i]] = (df[time_col[i + 1]] - df[time_col[i]]) // np.timedelta64(1, 's')
        col_diff.append(time_col[i + 1]+'-'+time_col[i])
    df_time[col_diff] = df_time[col_diff].fillna(0)
    df_time['session_len'] = df_time[col_diff].sum(axis=1)
    df_time['mean_session_len'] = df_time[col_diff].mean(axis=1)
    
    print('Обработка: смещения')
    df_time.loc[:,'time1_sec'] = df_time['time1_sec'] + 86399 - time_max
    df_time.loc[:,'time1_sec'] = df_time['time1_sec'].progress_apply(lambda x: x - 86399 + time_min if x > 86400 else x)
    
    df_time['month'] = df_time['month'].astype('object')
    df_time['weekday'] = df_time['weekday'].astype('object')
    
    df_time = pd.get_dummies(df_time)
    if not target:
        df_time['month_11'] = 0
        df_time['month_12'] = 0
        
    if target:
        df_time['target'] = df['target']
    
    df_time.drop(columns=col_diff, axis=1, inplace=True)
    df_time.drop(columns=time_col, axis=1, inplace=True)
    
    print('-'*50)
    print(' '*22 + 'READY')
    print('-'*50)
    
    return df_time

In [5]:
train_change = feature_engineering_time(train, time_col)
task_change = feature_engineering_time(task, time_col, target=False)

Обработка: mounth


  0%|          | 0/253561 [00:00<?, ?it/s]

Обработка: weekday


  0%|          | 0/253561 [00:00<?, ?it/s]

Обработка: time1


...

Обработка: Разницы


  0%|          | 0/9 [00:00<?, ?it/s]

Обработка: смещения


  0%|          | 0/82797 [00:00<?, ?it/s]

--------------------------------------------------
                      READY
--------------------------------------------------


In [6]:
train_change.head()

,time1_sec,session_len,mean_session_len,month_1,month_2,month_3,month_4,month_5,month_11,month_12,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,target
session_id,,,,,,,,,,,,,,,,,,
1,56510,0.0,0.000000,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,61135,26.0,2.888889,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,80362,7.0,0.777778,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
4,59477,270.0,30.000000,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
5,59530,246.0,27.333333,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


In [7]:
task_change.head()

,time1_sec,session_len,mean_session_len,month_1,month_2,month_3,month_4,month_5,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,month_11,month_12
session_id,,,,,,,,,,,,,,,,,
1,61138,7.0,0.777778,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,59973,85.0,9.444444,0,0,1,0,0,0,0,0,0,1,0,0,0,0
3,77657,84.0,9.333333,0,0,0,0,1,1,0,0,0,0,0,0,0,0
4,56544,4.0,0.444444,0,0,0,1,0,0,0,0,0,1,0,0,0,0
5,74676,13.0,1.444444,0,0,0,0,1,0,0,0,0,1,0,0,0,0


In [91]:
def get_auc_lr_valid_my(x, y, C=1.0, seed=42, ratio = 0.9, scaler=True):
    
    if scaler:
        ss = StandardScaler()
        x = ss.fit_transform(x)
    # Split the data into the training and validation sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=(1 - ratio),  random_state=seed)
    
    # Classifier training
    lr = LogisticRegression(C=C, random_state=seed, solver='lbfgs', max_iter=500).fit(x_train, y_train)
    
    # Prediction for validation set
    y_pred = lr.predict(x_test)
    y_pred_proba = lr.predict_proba(x_test)
    
    # Calculate the quality
    print('\n'+'*'*55)
    print(f'Roc_auc proba0: {roc_auc_score(y_test, y_pred_proba[:,0])}')
    print(f'Roc_auc proba1: {roc_auc_score(y_test, y_pred_proba[:,1])}')
    print('*'*55)
    print()
    print(classification_report(y_test, y_pred))
    
    return y_pred, y_pred_proba
    

In [92]:
tmp, tmp_proba = get_auc_lr_valid_my(train_change.drop('target', axis=1), train_change['target'])
pd.Series(tmp).value_counts()


*******************************************************
Roc_auc proba0: 0.102054516167777
Roc_auc proba1: 0.897945483832223
*******************************************************

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     25115
           1       0.00      0.00      0.00       242

    accuracy                           0.99     25357
   macro avg       0.50      0.50      0.50     25357
weighted avg       0.98      0.99      0.99     25357



C:\Users\79214\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\79214\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\79214\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0    25357
dtype: int64

---

sites

In [111]:
train[sites].fillna(0).astype('int').to_csv('src/train_sessions_text.txt', 
                                sep=' ', index=None, header=None)
task[sites].fillna(0).astype('int').to_csv('src/test_sessions_text.txt', 
                                sep=' ', index=None, header=None)

In [112]:
cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)
with open('src/train_sessions_text.txt') as inp_train_file:
    train_vector = cv.fit_transform(inp_train_file)
with open('src/test_sessions_text.txt') as inp_test_file:
    task_vector = cv.transform(inp_test_file)

In [113]:
lr = LogisticRegression(solver='lbfgs', max_iter=500).fit(train_vector, y_train['target'])
proba_vector = lr.predict_proba(train_vector)

In [130]:
train_change['proba_vec'] = lr.predict_proba(train_vector)[:,1]
task_change['proba_vec'] = lr.predict_proba(task_vector)[:,1]

In [131]:
def get_auc_lr_valid_my(x, y, ratio = 0.9, scaler=True):
    
    if scaler:
        ss = StandardScaler()
        x = ss.fit_transform(x)
    
    # Split the data into the training and validation sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=(1 - ratio),  random_state=17)
    
    # Classifier training
    lr = LogisticRegression(solver='lbfgs', max_iter=500).fit(x_train, y_train)
    
    # Prediction for validation set
    y_pred = lr.predict(x_test)
    y_pred_proba = lr.predict_proba(x_test)
    
    # Calculate the quality
    print('\n'+'*'*55)
    print(f'Roc_auc proba0: {roc_auc_score(y_test, y_pred_proba[:,0])}')
    print(f'Roc_auc proba1: {roc_auc_score(y_test, y_pred_proba[:,1])}')
    print('*'*55)
    print()
    print(classification_report(y_test, y_pred))
    
    return y_pred, y_pred_proba
    

In [133]:
tmp, tmp_proba = get_auc_lr_valid_my(train_change.drop('target', axis=1), train_change['target'])


*******************************************************
Roc_auc proba0: 0.007722371749260092
Roc_auc proba1: 0.99227762825074
*******************************************************

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     25141
           1       0.89      0.77      0.83       216

    accuracy                           1.00     25357
   macro avg       0.95      0.88      0.91     25357
weighted avg       1.00      1.00      1.00     25357



In [135]:
def pred_task(df_train, y, df_task):
    sc = StandardScaler()
    train = sc.fit_transform(df_train)
    tasc = sc.transform(df_task)
    lr = LogisticRegression(solver='lbfgs', max_iter=500).fit(df_train, y)
    y_pred = lr.predict_proba(df_task)[:, 1]
    
    return y_pred

In [137]:
y_pred = pred_task(train_change.drop('target', axis=1), y_train['target'], task_change)
predicted_df = pd.DataFrame({'target':y_pred}, index=task_change.index)
predicted_df

,target
session_id,
1,0.015815
2,0.011482
3,0.003559
4,0.021717
5,0.006252
...,...
82793,0.003068
82794,0.006650
82795,0.015608


In [141]:
predicted_df.to_csv('answers/baseline_4.csv')

---